## Pytorch Training Loop
<br>

- [Aula](https://www.youtube.com/watch?v=Dh9CL8fyG80&list=PLo2EIpI_JMQvWfQndUesu0nPBAtZ9gP1o&index=30)

In [ ]:
%%capture
!pip install datasets

from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint="bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenizer_function(exemples):
  return tokenizer(exemples["sentence1"], exemples["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenizer_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

data_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [ ]:
for batch in train_dataloader:
  break

print({k: v.shape for k, v in batch.items()})

In [ ]:
from transformers import AutoModelForSequenceClassification

checkpoint="bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)


In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

In [ ]:
%%capture
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
loss = outputs.loss
loss.backward()
optimizer.step()

optimizer.zero_grad()

In [ ]:
from transformers import get_scheduler

num_epochs=1
num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
%%capture
!pip install tqdm

from tqdm.auto import tqdm

In [ ]:
# progress_bar = tqdm(range(num_training_steps))
# model.train()

# for epoch in range(num_epochs):
#   for batch in train_dataloader:
#     bach = {k: v.shape for k, v in batch.items()}
#     outputs = model(**batch)
#     loss = outputs.loss
#     loss.backward()

#     optimizer.step()
#     lr_scheduler.step()
#     optimizer.zero_grad()

#     progress_bar.update(1)

In [ ]:
# from datasets import load_metric

# metric = load_metric("glue", "mrpc")
# model.eval()

# for batch in eval_dataloader:
#   batch = {k: v.to(device) for k, v in batch.items()}
#   with torch.no_grad():
#     outputs = model(**batch)

#   logits = outputs.logits
#   predictions = torch.argmax(logits, dim=-1)
#   metric.add_batch(predictions=predictions, references=batch["labels"])

# metric.compute()